In [1]:
import os
import numpy as np
import segyio
from scipy import stats
from multiprocessing import Pool

np.set_printoptions(threshold=np.inf) # Print the entire dataframe

from shutil import copyfile

# Data to use

In [2]:
# Seismic data: SEG-Y format
near_stack = os.path.join("..", "data", "NS2950-2180_3000-2230.sgy")
mid_stack = os.path.join("..", "data", "MS2950-2180_3000-2230.sgy")
far_stack = os.path.join("..", "data", "FS2950-2180_3000-2230.sgy")

# To store G and I
g_i = os.path.join("..", "data", "GradInter.sgy")
# Wells data: txt format
wells_path = '../data/wells_info.txt'

# Store the seismic data in a list
gather_path = [near_stack, mid_stack, far_stack]

# Input parameters to work with


In [3]:
# Name of the survery to display in the results
seismic_survey = 'POSEIDON 3D'

# Name and path of the merged SEG-Y file
merged_stacks = os.path.join("..", "data", "NMF2950-2180_3000-2230.sgy")

# Gradient file (SEGY)
gradient_segy = os.path.join("..", "data", "gradient.sgy")
# Intercept file (SEGY)
intercept_segy = os.path.join("..", "data", "intercept.sgy")
# Intercept file (SEGY)
rvalue_segy = os.path.join("..", "data", "correlation_c.sgy")
# Intercept file (SEGY)
pvalue_segy = os.path.join("..", "data", "pvalue.sgy")
# Intercept file (SEGY)
stderr_segy = os.path.join("..", "data", "standard_dev.sgy")

# Angle between the gathers given
angle_bet_gathers = 12

# Making a list with the angle of the gathers
angle_list = []
angle = angle_bet_gathers
for file in range(len(gather_path)):
    angle_list = angle_list + [angle]
    angle = angle + angle_bet_gathers

# Merging the partial angle stacks


In [4]:
def merging_stacks(list_of_gathers, merged_file_path, list_of_angles):
    
    """
    
    source1: https://github.com/equinor/segyio/blob/master/python/examples/make-file.py
    source2: https://github.com/equinor/segyio/blob/master/python/segyio/tracefield.py
    
    """
    
    # Validating the files
#    for file in list_of_gathers:
#        if file_validation(file) == True:
#            print(f"{file} has been validated")
#        else:
#            return(f"Unsuccessful validation: {file_validation(file)}")
    
    # Making the offset array
    offsts = np.array(list_of_angles)

    # Initializing the stacks
    with segyio.open(list_of_gathers[0]) as f, segyio.open(list_of_gathers[1]) as g, segyio.open(list_of_gathers[2]) as h:
        
        # Spec function to build the new segy
        spec = segyio.spec()
        spec.sorting = f.sorting
        spec.format = 1
        spec.samples = f.samples
        spec.ilines = f.ilines
        spec.xlines = f.xlines
        spec.offsets = offsts

        # Initializing the merged one
        with segyio.create(merged_file_path, spec) as s:
        
            # Index for the merge headers and trace
            merge_index = 0
            # Index for the original file's headers and trace
            stack_index = 0
            # For loop to set parameters according to the seismic lines and offset
            for il in spec.ilines:
                for xl in spec.xlines:
                    for offset in spec.offsets:
                        # Assigning headers [byte]
                        s.header[merge_index] = {segyio.su.tracl  : f.header[stack_index][1],
                                                 segyio.su.tracr  : f.header[stack_index][5],
                                                 segyio.su.fldr   : f.header[stack_index][9],
                                                 segyio.su.cdp    : f.header[stack_index][21],
                                                 segyio.su.cdpt   : f.header[stack_index][25],
                                                 segyio.su.offset : offset, # 37
                                                 segyio.su.scalco : f.header[stack_index][71],
                                                 segyio.su.ns     : f.header[stack_index][115],
                                                 segyio.su.dt     : f.header[stack_index][117],
                                                 segyio.su.cdpx   : f.header[stack_index][181],
                                                 segyio.su.cdpy   : f.header[stack_index][185],
                                                 segyio.su.iline  : il,  # 189
                                                 segyio.su.xline  : xl}  # 193
                        if offset == list_of_angles[0]:
                            stack = f # near
                        if offset == list_of_angles[1]:
                            stack = g # mid
                        if offset == list_of_angles[2]:
                            stack = h # far                   
                        
                        # Copying the amplitudes from each stack
                        s.trace[merge_index] = stack.trace[stack_index]
                        
                        merge_index += 1
                    stack_index += 1
                
    return (f"Successful merge. New SEG-Y file path: {merged_file_path}")

In [5]:
merging_stacks(gather_path, merged_stacks, angle_list)

'Successful merge. New SEG-Y file path: ../data/NMF2950-2180_3000-2230.sgy'

# First version of multiprocess avo attributes [26 min]

## Making the SEG-Y files to store the AVO attributes

In [6]:
def avo_storing_files(list_of_gathers, gradient_path, intercept_path, rvalue_path, pvalue_path, stderr_path): 
    # Initializing the stacks
    with segyio.open(list_of_gathers[0]) as f:
        
        # Spec function to build the new segy
        spec = segyio.tools.metadata(f)

        # Initializing the merged one
        with segyio.create(gradient_path, spec) as gradient:

            # Assigning headers [byte]
            gradient.text[0] = f.text[0]
            gradient.bin = f.bin
            gradient.header = f.header
            gradient.trace = f.trace
            
        print(f"Successful construction. Gradient path: ({gradient_path})")
            
    
    # Making copies of the new segy
    copyfile(gradient_path, intercept_path)
    print(f"Successful construction. Intercept path: ({intercept_path})")
    copyfile(gradient_path, rvalue_path)
    print(f"Successful construction. Correlation coeficient path: ({rvalue_path})")
    copyfile(gradient_path, pvalue_path)
    print(f"Successful construction. Pvalue path: ({pvalue_path})")
    copyfile(gradient_path, stderr_path)
    print(f"Successful construction. Standard Deviation path: ({stderr_path})")

    return

In [7]:
avo_storing_files(gather_path, gradient_segy, intercept_segy, rvalue_segy, pvalue_segy, stderr_segy)

Successful construction. Gradient path: (../data/gradient.sgy)
Successful construction. Intercept path: (../data/intercept.sgy)
Successful construction. Correlation coeficient path: (../data/correlation_c.sgy)
Successful construction. Pvalue path: (../data/pvalue.sgy)
Successful construction. Standard Deviation path: (../data/standard_dev.sgy)


## Function to give arguments to the pool

In [8]:
def input_generator(gather_file, list_of_angles, 
                    gradient_path, intercept_path, rvalue_path, pvalue_path, stderr_path):
    """
    
    """
    index = 0
    with segyio.open(gather_file, "r") as gather:
        for iline_number in gather.ilines:
            for xline_number in gather.xlines:
                    yield [gather_file, list_of_angles, iline_number, xline_number, index, 
                           gradient_path, intercept_path, rvalue_path, pvalue_path, stderr_path]
                    index += 1
#### test it
#input_args = input_generator(merged_stacks, angle_list, 
#                             gradient_segy, intercept_segy, rvalue_segy, pvalue_segy, stderr_segy)
#for row in input_args:
#    print(row)

## Function to compute the AVO attributes by trace

### First edition: lot of for loops

In [9]:
from scipy import stats
def  AVO_attributes_computation_1(gather_file, iline_number, xline_number, list_of_angles):
    """
    
    """
    # Initializing the sin(angle) list
    sins = []
    for angle in list_of_angles:
        sins += [np.sin(np.radians(angle)) * np.sin(np.radians(angle))]
    
    with segyio.open(gather_file, "r") as gather:
        for time_slice in range(len(gather.samples)):
            amp = []
            for angle in list_of_angles:
                amp += [gather.gather[iline_number, xline_number, angle][time_slice]]
            
            # Regression
            gradient, intercept, r_value, p_value, std_error = stats.linregress(sins, amp)
            
            yield[intercept, gradient, r_value, p_value, std_error]

# Test it
#avo_attributes = AVO_attributes_computation(merged_stacks, 2950, 2180, angle_list)
#
#for row in avo_attributes:
#
#    print(row)

## Function to store the AVO attributes by trace

In [10]:
def store_AVO_attributes(generator_list):
    """
    
    """
    # Parameters to use in the functions
    gather_file = generator_list[0]
    list_of_angles = generator_list[1]
    iline_number = generator_list[2]
    xline_number = generator_list[3]
    index = generator_list[4]
    intercept_file = generator_list[5]
    gradient_file = generator_list[6]
    rvalue_file = generator_list[7]
    pvalue_file = generator_list[8]
    stderr_file = generator_list[9]
    
    to_store = AVO_attributes_computation_1(gather_file, iline_number, xline_number, 
                                          list_of_angles)
    cont = 0
    with segyio.open(gradient_file, "r+") as grad:
        gradient = np.zeros([len(grad.samples), ], dtype = "float32")
        intercept = np.zeros([len(grad.samples), ], dtype = "float32")
        rvalue = np.zeros([len(grad.samples), ], dtype = "float32")#Correlation coefficient.
        pvalue = np.zeros([len(grad.samples), ], dtype = "float32")
        stderr = np.zeros([len(grad.samples), ], dtype = "float32") #Standard error of the estimated gradient.

        # Inspecting the generator
        for attribute in to_store:
            gradient[cont] = attribute[0]
            intercept[cont] = attribute[1]
            rvalue[cont] = attribute[2]
            pvalue[cont] = attribute[3]
            stderr[cont] = attribute[4]
            cont += 1
        
        # storing
        grad.trace[index] = gradient  

    with segyio.open(intercept_file, "r+") as intercept_segy:
        intercept_segy.trace[index] = intercept

    with segyio.open(rvalue_file, "r+") as rvalue_segy:
        rvalue_segy.trace[index] = rvalue

    with segyio.open(pvalue_file, "r+") as pvalue_segy:
        pvalue_segy.trace[index] = pvalue

    with segyio.open(stderr_file, "r+") as stderr_segy:
        stderr_segy.trace[index] = stderr
    
    print(f"AVO attributes for trace [{iline_number},{xline_number}, {index}] has been stored successfully")
    return
    
# Test and Taste
#input_args = input_generator(merged_stacks, angle_list, 
#                             gradient_segy, intercept_segy, rvalue_segy, pvalue_segy, stderr_segy)
#
#for row in input_args:
#    print(store_AVO_attributes(row))

input_args = input_generator(merged_stacks, angle_list, 
                             gradient_segy, intercept_segy, rvalue_segy, pvalue_segy, stderr_segy)

for row in input_args:
    print(store_AVO_attributes(row))

## Integrating the previous functions [MP]

In [11]:
def avo_storeputation(gather_file, list_of_angles, 
                      gradient_file, intercept_file, rvalue_file, pvalue_file, stderr_file):
    
    input_data = input_generator(gather_file, list_of_angles, 
                                 gradient_file, intercept_file, rvalue_file, pvalue_file, stderr_file)
    
    if __name__ == "__main__":
        p = Pool(maxtasksperchild=1)
        p.map(store_AVO_attributes, input_data, chunksize=1)

In [12]:
avo_storeputation(merged_stacks, angle_list,
                  gradient_segy, intercept_segy, rvalue_segy, pvalue_segy, stderr_segy)

AVO attributes for trace [2950,2180, 0] has been stored successfully
AVO attributes for trace [2950,2182, 2] has been stored successfully
AVO attributes for trace [2950,2181, 1] has been stored successfully
AVO attributes for trace [2950,2183, 3] has been stored successfully
AVO attributes for trace [2950,2184, 4] has been stored successfully
AVO attributes for trace [2950,2186, 6] has been stored successfully
AVO attributes for trace [2950,2185, 5] has been stored successfully
AVO attributes for trace [2950,2187, 7] has been stored successfully
AVO attributes for trace [2950,2188, 8] has been stored successfully
AVO attributes for trace [2950,2190, 10] has been stored successfully
AVO attributes for trace [2950,2189, 9] has been stored successfully
AVO attributes for trace [2950,2191, 11] has been stored successfully
AVO attributes for trace [2950,2192, 12] has been stored successfully
AVO attributes for trace [2950,2193, 13] has been stored successfully
AVO attributes for trace [2950

AVO attributes for trace [2952,2196, 118] has been stored successfully
AVO attributes for trace [2952,2195, 117] has been stored successfully
AVO attributes for trace [2952,2197, 119] has been stored successfully
AVO attributes for trace [2952,2198, 120] has been stored successfully
AVO attributes for trace [2952,2199, 121] has been stored successfully
AVO attributes for trace [2952,2200, 122] has been stored successfully
AVO attributes for trace [2952,2201, 123] has been stored successfully
AVO attributes for trace [2952,2202, 124] has been stored successfully
AVO attributes for trace [2952,2203, 125] has been stored successfully
AVO attributes for trace [2952,2204, 126] has been stored successfully
AVO attributes for trace [2952,2205, 127] has been stored successfully
AVO attributes for trace [2952,2206, 128] has been stored successfully
AVO attributes for trace [2952,2207, 129] has been stored successfully
AVO attributes for trace [2952,2208, 130] has been stored successfully
AVO at

AVO attributes for trace [2954,2209, 233] has been stored successfully
AVO attributes for trace [2954,2210, 234] has been stored successfully
AVO attributes for trace [2954,2211, 235] has been stored successfully
AVO attributes for trace [2954,2212, 236] has been stored successfully
AVO attributes for trace [2954,2213, 237] has been stored successfully
AVO attributes for trace [2954,2214, 238] has been stored successfully
AVO attributes for trace [2954,2215, 239] has been stored successfully
AVO attributes for trace [2954,2216, 240] has been stored successfully
AVO attributes for trace [2954,2217, 241] has been stored successfully
AVO attributes for trace [2954,2218, 242] has been stored successfully
AVO attributes for trace [2954,2219, 243] has been stored successfully
AVO attributes for trace [2954,2220, 244] has been stored successfully
AVO attributes for trace [2954,2221, 245] has been stored successfully
AVO attributes for trace [2954,2222, 246] has been stored successfully
AVO at

AVO attributes for trace [2956,2223, 349] has been stored successfully
AVO attributes for trace [2956,2224, 350] has been stored successfully
AVO attributes for trace [2956,2225, 351] has been stored successfully
AVO attributes for trace [2956,2226, 352] has been stored successfully
AVO attributes for trace [2956,2227, 353] has been stored successfully
AVO attributes for trace [2956,2228, 354] has been stored successfully
AVO attributes for trace [2956,2229, 355] has been stored successfully
AVO attributes for trace [2956,2230, 356] has been stored successfully
AVO attributes for trace [2957,2180, 357] has been stored successfully
AVO attributes for trace [2957,2181, 358] has been stored successfully
AVO attributes for trace [2957,2182, 359] has been stored successfully
AVO attributes for trace [2957,2183, 360] has been stored successfully
AVO attributes for trace [2957,2184, 361] has been stored successfully
AVO attributes for trace [2957,2185, 362] has been stored successfully
AVO at

AVO attributes for trace [2959,2186, 465] has been stored successfully
AVO attributes for trace [2959,2187, 466] has been stored successfully
AVO attributes for trace [2959,2188, 467] has been stored successfully
AVO attributes for trace [2959,2189, 468] has been stored successfully
AVO attributes for trace [2959,2190, 469] has been stored successfully
AVO attributes for trace [2959,2191, 470] has been stored successfully
AVO attributes for trace [2959,2192, 471] has been stored successfully
AVO attributes for trace [2959,2193, 472] has been stored successfully
AVO attributes for trace [2959,2194, 473] has been stored successfully
AVO attributes for trace [2959,2195, 474] has been stored successfully
AVO attributes for trace [2959,2196, 475] has been stored successfully
AVO attributes for trace [2959,2197, 476] has been stored successfully
AVO attributes for trace [2959,2199, 478] has been stored successfully
AVO attributes for trace [2959,2198, 477] has been stored successfully
AVO at

...

AVO attributes for trace [2990,2210, 2070] has been stored successfully
AVO attributes for trace [2990,2211, 2071] has been stored successfully
AVO attributes for trace [2990,2212, 2072] has been stored successfully
AVO attributes for trace [2990,2213, 2073] has been stored successfully
AVO attributes for trace [2990,2214, 2074] has been stored successfully
AVO attributes for trace [2990,2215, 2075] has been stored successfully
AVO attributes for trace [2990,2216, 2076] has been stored successfully
AVO attributes for trace [2990,2217, 2077] has been stored successfully
AVO attributes for trace [2990,2218, 2078] has been stored successfully
AVO attributes for trace [2990,2219, 2079] has been stored successfully
AVO attributes for trace [2990,2220, 2080] has been stored successfully
AVO attributes for trace [2990,2221, 2081] has been stored successfully
AVO attributes for trace [2990,2222, 2082] has been stored successfully
AVO attributes for trace [2990,2223, 2083] has been stored succe

AVO attributes for trace [2992,2222, 2184] has been stored successfully
AVO attributes for trace [2992,2223, 2185] has been stored successfully
AVO attributes for trace [2992,2224, 2186] has been stored successfully
AVO attributes for trace [2992,2225, 2187] has been stored successfully
AVO attributes for trace [2992,2226, 2188] has been stored successfully
AVO attributes for trace [2992,2227, 2189] has been stored successfully
AVO attributes for trace [2992,2228, 2190] has been stored successfully
AVO attributes for trace [2992,2229, 2191] has been stored successfully
AVO attributes for trace [2992,2230, 2192] has been stored successfully
AVO attributes for trace [2993,2180, 2193] has been stored successfully
AVO attributes for trace [2993,2181, 2194] has been stored successfully
AVO attributes for trace [2993,2182, 2195] has been stored successfully
AVO attributes for trace [2993,2183, 2196] has been stored successfully
AVO attributes for trace [2993,2184, 2197] has been stored succe

AVO attributes for trace [2995,2182, 2297] has been stored successfully
AVO attributes for trace [2995,2184, 2299] has been stored successfully
AVO attributes for trace [2995,2185, 2300] has been stored successfully
AVO attributes for trace [2995,2186, 2301] has been stored successfully
AVO attributes for trace [2995,2187, 2302] has been stored successfully
AVO attributes for trace [2995,2188, 2303] has been stored successfully
AVO attributes for trace [2995,2189, 2304] has been stored successfully
AVO attributes for trace [2995,2190, 2305] has been stored successfully
AVO attributes for trace [2995,2191, 2306] has been stored successfully
AVO attributes for trace [2995,2192, 2307] has been stored successfully
AVO attributes for trace [2995,2193, 2308] has been stored successfully
AVO attributes for trace [2995,2194, 2309] has been stored successfully
AVO attributes for trace [2995,2195, 2310] has been stored successfully
AVO attributes for trace [2995,2196, 2311] has been stored succe

AVO attributes for trace [2997,2195, 2412] has been stored successfully
AVO attributes for trace [2997,2196, 2413] has been stored successfully
AVO attributes for trace [2997,2197, 2414] has been stored successfully
AVO attributes for trace [2997,2198, 2415] has been stored successfully
AVO attributes for trace [2997,2199, 2416] has been stored successfully
AVO attributes for trace [2997,2200, 2417] has been stored successfully
AVO attributes for trace [2997,2201, 2418] has been stored successfully
AVO attributes for trace [2997,2202, 2419] has been stored successfully
AVO attributes for trace [2997,2203, 2420] has been stored successfully
AVO attributes for trace [2997,2204, 2421] has been stored successfully
AVO attributes for trace [2997,2205, 2422] has been stored successfully
AVO attributes for trace [2997,2206, 2423] has been stored successfully
AVO attributes for trace [2997,2207, 2424] has been stored successfully
AVO attributes for trace [2997,2208, 2425] has been stored succe

AVO attributes for trace [2999,2207, 2526] has been stored successfully
AVO attributes for trace [2999,2208, 2527] has been stored successfully
AVO attributes for trace [2999,2209, 2528] has been stored successfully
AVO attributes for trace [2999,2210, 2529] has been stored successfully
AVO attributes for trace [2999,2211, 2530] has been stored successfully
AVO attributes for trace [2999,2212, 2531] has been stored successfully
AVO attributes for trace [2999,2213, 2532] has been stored successfully
AVO attributes for trace [2999,2214, 2533] has been stored successfully
AVO attributes for trace [2999,2215, 2534] has been stored successfully
AVO attributes for trace [2999,2216, 2535] has been stored successfully
AVO attributes for trace [2999,2217, 2536] has been stored successfully
AVO attributes for trace [2999,2218, 2537] has been stored successfully
AVO attributes for trace [2999,2219, 2538] has been stored successfully
AVO attributes for trace [2999,2220, 2539] has been stored succe

## testing new files

In [11]:
with segyio.open(stderr_segy, "r+") as f:
    print(f.trace[0])

[   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    1.
    0.    0.    2.    1.   -4.   -2.    5.    1.   -2.    4.    3.    2.
    4.   -5.   12.   54.   12.  -92.  -86.   11.   

# Second version of multiprocess avo attributes [ 21 min]

## Making the SEG-Y files to store the AVO attributes

## Function to give arguments to the pool

## Function to compute the AVO attributes by trace

### Second edition: vectorized function
the function below computes and stores the data because of this hypothesis: if two functions are applied, the one receiving the data could represent a lost of ram!!   


In [12]:
def  AVO_attributes_computation_2(generator_list):
    
    """
    
    """
    # Parameters to use in the functions
    gather_file = generator_list[0]
    list_of_angles = generator_list[1]
    iline_number = generator_list[2]
    xline_number = generator_list[3]
    index = generator_list[4]
    intercept_file = generator_list[5]
    gradient_file = generator_list[6]
    rvalue_file = generator_list[7]
    pvalue_file = generator_list[8]
    stderr_file = generator_list[9]
    
    # Initializing the sin(angle) list
    sins = []
    
    # Opening the gather to: 
                            #1) Create an empty array to store the values within the gather file
                            #2) Make multiple arrays to be used by segyio's trace function
    with segyio.open(gather_file, "r") as gather:
        gradient = np.zeros([len(gather.samples), ], dtype = "float32")
        intercept = np.zeros([len(gather.samples), ], dtype = "float32")
        rvalue = np.zeros([len(gather.samples), ], dtype = "float32")#Correlation coefficient.
        pvalue = np.zeros([len(gather.samples), ], dtype = "float32")
        stderr = np.zeros([len(gather.samples), ], dtype = "float32") #Standard error of the estimated gradient.
        
        # For loot to append the values for sins and to store the amplitudes in an array(3,1501)
        for angle in list_of_angles:
            sins += [np.sin(np.radians(angle)) * np.sin(np.radians(angle))]
            
            # If the angle is the first, then initialize the amp array
            if angle == list_of_angles[0]:
                amp = gather.gather[iline_number, xline_number, angle].reshape(len(gather.samples), 1)
            
            # If it's not the first angle in the list, concatenate to an existing variable (amp)
            else:
                amp = np.concatenate((amp, 
                                      gather.gather[iline_number, xline_number, angle].reshape(len(gather.samples), 
        
                                                                                               1)),axis=1)
        # the amp is was reshaped to (3, 1501) in order to introduce the amplitudes of a slice in the
        # Scipy function: has been VECTORIZED!!
        
        # For loop to select and to introduce values to Scipy's
        for row in range(amp.shape[0]):
            gradient[row], intercept[row], rvalue[row], pvalue[row], stderr[row] = stats.linregress(sins, amp[row])
        
    # Storing the data: Opening and closing the segy's. This should improve the perfomance of the function
    with segyio.open(gradient_file, "r+") as gradient_segy:
        gradient_segy.trace[index] = gradient

    with segyio.open(intercept_file, "r+") as intercept_segy:
        intercept_segy.trace[index] = intercept

    with segyio.open(rvalue_file, "r+") as rvalue_segy:
        rvalue_segy.trace[index] = rvalue

    with segyio.open(pvalue_file, "r+") as pvalue_segy:
        pvalue_segy.trace[index] = pvalue

    with segyio.open(stderr_file, "r+") as stderr_segy:
            stderr_segy.trace[index] = stderr
    
    print(f"AVO attributes for trace number {index}, [{iline_number},{xline_number}], has been stored successfully")
    return

#test it
#AVO_attributes_computation_2(merged_stacks, 2950, 2180, angle_list, 0,
#                             gradient_segy, intercept_segy, rvalue_segy, pvalue_segy, stderr_segy)
## TASTE it (2)
#input_args = input_generator(merged_stacks, angle_list, 
#                             gradient_segy, intercept_segy, rvalue_segy, pvalue_segy, stderr_segy)
#
#for row in input_args:
#    print(AVO_attributes_computation_2(row))

## Integrating the previous functions [MP]

In [13]:
def avo_storeputation(gather_file, list_of_angles, 
                      gradient_file, intercept_file, rvalue_file, pvalue_file, stderr_file):
    
    input_args = input_generator(merged_stacks, angle_list, 
                             gradient_segy, intercept_segy, rvalue_segy, pvalue_segy, stderr_segy)
    
    if __name__ == "__main__":
        p = Pool(4, maxtasksperchild=1)
        p.map(AVO_attributes_computation_2, input_args, chunksize=1)

In [14]:
avo_storeputation(merged_stacks, angle_list,
                  gradient_segy, intercept_segy, rvalue_segy, pvalue_segy, stderr_segy)

AVO attributes for trace number 0, [2950,2180], has been stored successfully
AVO attributes for trace number 1, [2950,2181], has been stored successfully
AVO attributes for trace number 2, [2950,2182], has been stored successfully
AVO attributes for trace number 3, [2950,2183], has been stored successfully
AVO attributes for trace number 4, [2950,2184], has been stored successfully
AVO attributes for trace number 5, [2950,2185], has been stored successfully
AVO attributes for trace number 6, [2950,2186], has been stored successfully
AVO attributes for trace number 7, [2950,2187], has been stored successfully
AVO attributes for trace number 8, [2950,2188], has been stored successfully
AVO attributes for trace number 10, [2950,2190], has been stored successfully
AVO attributes for trace number 9, [2950,2189], has been stored successfully
AVO attributes for trace number 11, [2950,2191], has been stored successfully
AVO attributes for trace number 12, [2950,2192], has been stored successfu

AVO attributes for trace number 106, [2952,2184], has been stored successfully
AVO attributes for trace number 107, [2952,2185], has been stored successfully
AVO attributes for trace number 108, [2952,2186], has been stored successfully
AVO attributes for trace number 109, [2952,2187], has been stored successfully
AVO attributes for trace number 110, [2952,2188], has been stored successfully
AVO attributes for trace number 111, [2952,2189], has been stored successfully
AVO attributes for trace number 112, [2952,2190], has been stored successfully
AVO attributes for trace number 113, [2952,2191], has been stored successfully
AVO attributes for trace number 114, [2952,2192], has been stored successfully
AVO attributes for trace number 115, [2952,2193], has been stored successfully
AVO attributes for trace number 116, [2952,2194], has been stored successfully
AVO attributes for trace number 117, [2952,2195], has been stored successfully
AVO attributes for trace number 118, [2952,2196], ha

AVO attributes for trace number 210, [2954,2186], has been stored successfully
AVO attributes for trace number 211, [2954,2187], has been stored successfully
AVO attributes for trace number 212, [2954,2188], has been stored successfully
AVO attributes for trace number 213, [2954,2189], has been stored successfully
AVO attributes for trace number 214, [2954,2190], has been stored successfully
AVO attributes for trace number 215, [2954,2191], has been stored successfully
AVO attributes for trace number 216, [2954,2192], has been stored successfully
AVO attributes for trace number 217, [2954,2193], has been stored successfully
AVO attributes for trace number 218, [2954,2194], has been stored successfully
AVO attributes for trace number 219, [2954,2195], has been stored successfully
AVO attributes for trace number 220, [2954,2196], has been stored successfully
AVO attributes for trace number 221, [2954,2197], has been stored successfully
AVO attributes for trace number 222, [2954,2198], ha

AVO attributes for trace number 314, [2956,2188], has been stored successfully
AVO attributes for trace number 315, [2956,2189], has been stored successfully
AVO attributes for trace number 316, [2956,2190], has been stored successfully
AVO attributes for trace number 317, [2956,2191], has been stored successfully
AVO attributes for trace number 318, [2956,2192], has been stored successfully
AVO attributes for trace number 319, [2956,2193], has been stored successfully
AVO attributes for trace number 320, [2956,2194], has been stored successfully
AVO attributes for trace number 321, [2956,2195], has been stored successfully
AVO attributes for trace number 322, [2956,2196], has been stored successfully
AVO attributes for trace number 324, [2956,2198], has been stored successfully
AVO attributes for trace number 323, [2956,2197], has been stored successfully
AVO attributes for trace number 325, [2956,2199], has been stored successfully
AVO attributes for trace number 326, [2956,2200], ha

AVO attributes for trace number 418, [2958,2190], has been stored successfully
AVO attributes for trace number 419, [2958,2191], has been stored successfully
AVO attributes for trace number 420, [2958,2192], has been stored successfully
AVO attributes for trace number 421, [2958,2193], has been stored successfully
AVO attributes for trace number 422, [2958,2194], has been stored successfully
AVO attributes for trace number 423, [2958,2195], has been stored successfully
AVO attributes for trace number 424, [2958,2196], has been stored successfully
AVO attributes for trace number 425, [2958,2197], has been stored successfully
AVO attributes for trace number 426, [2958,2198], has been stored successfully
AVO attributes for trace number 427, [2958,2199], has been stored successfully
AVO attributes for trace number 428, [2958,2200], has been stored successfully
AVO attributes for trace number 429, [2958,2201], has been stored successfully
AVO attributes for trace number 430, [2958,2202], ha

...

AVO attributes for trace number 2175, [2992,2213], has been stored successfully
AVO attributes for trace number 2176, [2992,2214], has been stored successfully
AVO attributes for trace number 2177, [2992,2215], has been stored successfully
AVO attributes for trace number 2178, [2992,2216], has been stored successfully
AVO attributes for trace number 2179, [2992,2217], has been stored successfully
AVO attributes for trace number 2180, [2992,2218], has been stored successfully
AVO attributes for trace number 2181, [2992,2219], has been stored successfully
AVO attributes for trace number 2182, [2992,2220], has been stored successfully
AVO attributes for trace number 2183, [2992,2221], has been stored successfully
AVO attributes for trace number 2184, [2992,2222], has been stored successfully
AVO attributes for trace number 2185, [2992,2223], has been stored successfully
AVO attributes for trace number 2186, [2992,2224], has been stored successfully
AVO attributes for trace number 2187, [2

AVO attributes for trace number 2278, [2994,2214], has been stored successfully
AVO attributes for trace number 2279, [2994,2215], has been stored successfully
AVO attributes for trace number 2280, [2994,2216], has been stored successfully
AVO attributes for trace number 2281, [2994,2217], has been stored successfully
AVO attributes for trace number 2282, [2994,2218], has been stored successfully
AVO attributes for trace number 2283, [2994,2219], has been stored successfully
AVO attributes for trace number 2284, [2994,2220], has been stored successfully
AVO attributes for trace number 2285, [2994,2221], has been stored successfully
AVO attributes for trace number 2286, [2994,2222], has been stored successfully
AVO attributes for trace number 2287, [2994,2223], has been stored successfully
AVO attributes for trace number 2288, [2994,2224], has been stored successfully
AVO attributes for trace number 2289, [2994,2225], has been stored successfully
AVO attributes for trace number 2290, [2

AVO attributes for trace number 2381, [2996,2215], has been stored successfully
AVO attributes for trace number 2382, [2996,2216], has been stored successfully
AVO attributes for trace number 2383, [2996,2217], has been stored successfully
AVO attributes for trace number 2384, [2996,2218], has been stored successfully
AVO attributes for trace number 2385, [2996,2219], has been stored successfully
AVO attributes for trace number 2386, [2996,2220], has been stored successfully
AVO attributes for trace number 2387, [2996,2221], has been stored successfully
AVO attributes for trace number 2388, [2996,2222], has been stored successfully
AVO attributes for trace number 2389, [2996,2223], has been stored successfully
AVO attributes for trace number 2390, [2996,2224], has been stored successfully
AVO attributes for trace number 2391, [2996,2225], has been stored successfully
AVO attributes for trace number 2392, [2996,2226], has been stored successfully
AVO attributes for trace number 2393, [2

AVO attributes for trace number 2484, [2998,2216], has been stored successfully
AVO attributes for trace number 2485, [2998,2217], has been stored successfully
AVO attributes for trace number 2486, [2998,2218], has been stored successfully
AVO attributes for trace number 2487, [2998,2219], has been stored successfully
AVO attributes for trace number 2488, [2998,2220], has been stored successfully
AVO attributes for trace number 2489, [2998,2221], has been stored successfully
AVO attributes for trace number 2490, [2998,2222], has been stored successfully
AVO attributes for trace number 2491, [2998,2223], has been stored successfully
AVO attributes for trace number 2492, [2998,2224], has been stored successfully
AVO attributes for trace number 2493, [2998,2225], has been stored successfully
AVO attributes for trace number 2494, [2998,2226], has been stored successfully
AVO attributes for trace number 2495, [2998,2227], has been stored successfully
AVO attributes for trace number 2496, [2

AVO attributes for trace number 2587, [3000,2217], has been stored successfully
AVO attributes for trace number 2588, [3000,2218], has been stored successfully
AVO attributes for trace number 2589, [3000,2219], has been stored successfully
AVO attributes for trace number 2590, [3000,2220], has been stored successfully
AVO attributes for trace number 2591, [3000,2221], has been stored successfully
AVO attributes for trace number 2592, [3000,2222], has been stored successfully
AVO attributes for trace number 2593, [3000,2223], has been stored successfully
AVO attributes for trace number 2594, [3000,2224], has been stored successfully
AVO attributes for trace number 2595, [3000,2225], has been stored successfully
AVO attributes for trace number 2596, [3000,2226], has been stored successfully
AVO attributes for trace number 2597, [3000,2227], has been stored successfully
AVO attributes for trace number 2598, [3000,2228], has been stored successfully
AVO attributes for trace number 2600, [3

# DelayRecordingTime: 0
- cube size
- chunks, maxtask
- store the other 3 scipy's parameters
- vectorize the for loops